In [1]:
import sys
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import dash

In [2]:
if "/home/alex/qi3/hmmtuf" not in sys.path:
    sys.path.append("/home/alex/qi3/hmmtuf")

In [3]:
from db.sqlite3_db_connector import SQLiteDBConnector
from compute_engine.src.utils import INFO

In [4]:
# connect to the DB
db_connector = SQLiteDBConnector(db_file="../../play_ground.sqlite3")
db_connector.connect()

In [5]:
# check the table names
tbl_names = db_connector.get_table_names()

for name in tbl_names:
    print("{0} DB table: {1}".format(INFO, name))

INFO: DB table: distance_metric_type
INFO: DB table: distance_sequence_type
INFO: DB table: repeats
INFO: DB table: hmm_state_types
INFO: DB table: repeats_distances


In [6]:
uique_seq_types = [(0, "Select")]

sql = "SELECT * from distance_sequence_type"
rows = db_connector.fetch_all(sql=sql)

for item in rows:
    uique_seq_types.append(item)
    print("{0} Sequence type {1}".format(INFO, item))

INFO: Sequence type (1, 'NORMAL')
INFO: Sequence type (2, 'PURINE')
INFO: Sequence type (3, 'AMINO')
INFO: Sequence type (4, 'WEAK_HYDROGEN')


In [7]:
uique_dist_types = [(0, "Select")]

sql = "SELECT * from distance_metric_type"
rows = db_connector.fetch_all(sql=sql)

for item in rows:
    uique_dist_types.append(item)
    print("{0} Metric type {1}".format(INFO, item))

INFO: Metric type (1, 'Bag', 'bag')
INFO: Metric type (2, 'Cosine', 'cos')
INFO: Metric type (3, 'DamerauLevenshtein', 'damlev')
INFO: Metric type (4, 'Gotoh', 'got')
INFO: Metric type (5, 'Hamming', 'ham')
INFO: Metric type (6, 'Jaccard', 'jac')
INFO: Metric type (7, 'JaroWinkler', 'jwink')
INFO: Metric type (8, 'LCSSeq', 'lcsseq')
INFO: Metric type (9, 'LCSStr', 'lcsstr')
INFO: Metric type (10, 'Levenshtein', 'lev')
INFO: Metric type (11, 'MLIPNS', 'mlipns')
INFO: Metric type (12, 'MongeElkan', 'mon')
INFO: Metric type (13, 'NeedlemanWunsch', 'nw')
INFO: Metric type (14, 'Overlap', 'ov')
INFO: Metric type (15, 'Sorensen', 'sor')
INFO: Metric type (16, 'StrCmp95', 'str')
INFO: Metric type (17, 'SmithWaterman', 'sw')
INFO: Metric type (18, 'Tanimoto', 'tan')
INFO: Metric type (19, 'Tversky', 'tve')


In [8]:
unique_state_types = [(0, "Select")]
sql = "SELECT * from hmm_state_types"
rows = db_connector.fetch_all(sql=sql)

for item in rows:
    unique_state_types.append(item)
    print("{0} State type {1}".format(INFO, item))

INFO: State type (1, 'NORMAL')
INFO: State type (2, 'TUF')
INFO: State type (3, 'DELETION')
INFO: State type (4, 'DUPLICATION')


In [9]:
unique_state_types.remove((4, 'DUPLICATION'))
for i, item in enumerate(unique_state_types):
    if item == (3, 'DELETION'):
        unique_state_types[i] = (3, 'CORE')

for item in unique_state_types:
    print("{0} State type {1}".format(INFO, item))

INFO: State type (0, 'Select')
INFO: State type (1, 'NORMAL')
INFO: State type (2, 'TUF')
INFO: State type (3, 'CORE')


In [10]:
db_connector.delete_table(tbl_name="temp.temp_repeats")

In [11]:
sql = '''CREATE TEMPORARY TABLE temp_repeats AS SELECT'''
sql += ''' id FROM repeats where gc < {0} AND repeat_seq != 'NO_REPEATS';'''.format(0.5)
db_connector.create_tmp_table(sql=sql)

In [12]:
# the application
app = JupyterDash(__name__)

In [13]:
print(uique_seq_types)

[(0, 'Select'), (1, 'NORMAL'), (2, 'PURINE'), (3, 'AMINO'), (4, 'WEAK_HYDROGEN')]


In [14]:
gc_limit_type = [(0, "Select"), (1, "AVG"), (2, "MIN"), (3, "MAX")]
gc_limit = [(0, "Select"), (1, "<"), (2, ">"), (3, "<="), (4, ">="), (5, "=")]
gc_limit_map = {0: "Select", 1: "<", 2: ">", 3: "<=", 4: ">=", 5: "="}

In [15]:
app.layout = html.Div([
    html.H1("Distances Plot"),
    html.H3("Sequence type"),
    dcc.Dropdown(
        id="dropdown-sequence",
        options=[{"label": x[1], "value": x[0]} for x in uique_seq_types],
        value=uique_seq_types[0][0],
        clearable=False,
    ),
    
    html.H3("Distance type"),
    dcc.Dropdown(
        id="dropdown-distance",
        options=[{"label": x[1], "value": x[0]} for x in uique_dist_types],
        value=uique_dist_types[0][0],
        clearable=False,
    ),
    
    html.H3("GC Limit Variable"),
    dcc.Dropdown(
        id="dropdown-gc-limit-type",
        options=[{"label": x[1], "value": x[0]} for x in gc_limit_type],
        value=gc_limit_type[0][0],
        clearable=False,
    ),
    
    html.H3("GC Limiter"),
    dcc.Dropdown(
        id="dropdown-gc-limit",
        options=[{"label": x[1], "value": x[0]} for x in gc_limit],
        value=gc_limit[0][0],
        clearable=False,
    ),
    
    html.H3("GC Value"),
    dcc.Input(
        id="gc-limit-value",
        type="number",
    ),
    html.Br(id="break-id-1"),
    html.Br(id="break-id-2"),
    html.Button(children ='Compute', id='compute-btn', n_clicks =0),
     html.Br(id="break-id-3"),
    html.Br(id="break-id-4"),
    html.Div(children=[html.H5("Error Messages"), html.Div(id="error-messages-id")]),
     html.Br(id="break-id-5"),
    html.Br(id="break-id-6"),
    html.H3("Normal state"),
    html.Div(children=[html.H5("Number of sequences"), html.Div(id="normal-n-distances")]),
    dcc.Graph(id="normal-bar-chart"),
    html.H3("TUF state"),
    html.Div(children=[html.H5("Number of sequences"), html.Div(id="tuf-n-distances")]),
    dcc.Graph(id="tuf-bar-chart"),
    html.H3("Core"),
    html.Div(children=[html.H5("Number of sequences"), html.Div(id="core-n-distances")] ),
    dcc.Graph(id="core-bar-chart"),
    
])

In [16]:
def create_plot_from_state_type(state_type_id, metric_type_id, sequence_type_id):
    
    # we don't use any limiters
    sql = "SELECT value FROM repeats_distances WHERE hmm_state_id_1 = {0}".format(state_type_id) 
    sql += " AND  hmm_state_id_2 = {0}".format(state_type_id)
    sql += " AND metric_type_id={0} AND sequence_type_id={1} LIMIT 10".format(metric_type_id, 
                                                                              sequence_type_id)
    
    error_message = ""
         
    try:
        print("{0} Executing sql={1}".format(INFO, sql))
        local_db = SQLiteDBConnector(db_file="../../play_ground.sqlite3")
        local_db.connect()
        rows = local_db.fetch_all(sql=sql)
        print("{0} Fetched number of rows={1}".format(INFO, len(rows)))
    except Exception as e:
        rows = []
        error_message = str(e)
    
    
    counts, bins = np.histogram(rows, bins=35)
    bins = 0.5 * (bins[:-1] + bins[1:])

    fig = px.bar(x=bins, y=counts, orientation='v', 
                 labels={'x':'distance', 'y':'count'}, range_x=[0,1])
    fig.update_layout(xaxis = dict(
            tickmode = 'linear',
            tick0 = 0.0,
            dtick = 0.15))

    return error_message, fig, len(rows)
    

In [17]:


def create_figure_plot(state_type_id, metric_type_id, 
                       sequence_type_id, gc_limit_type, gc_limiter, gc_value, btn_clicks):
    
    
    if btn_clicks == 0:
        
        # nothing to compute
        rows = []
        error_message = ""
        
    elif btn_clicks !=0 and sequence_type_id == 0 or metric_type_id == 0:
        # we have nothing to plot
        rows = []
        error_message = ""
    else:
        
        if gc_limit_type == 0 and gc_limiter == 0 and gc_value is None:
            
            # we don't want GC limiters
            return create_plot_from_state_type(state_type_id=state_type_id, 
                                                   metric_type_id=metric_type_id, 
                                                   sequence_type_id=sequence_type_id)
        else:
            
            ## error handling
            if gc_limit_type != 0 and gc_limiter == 0 or gc_value is None:
                rows = []
                error_message = " GC limit variable specified but" 
                error_message += " not the GC limiter direction or the GC value."
            elif gc_limiter != 0 and gc_limit_type == 0 or gc_value is None:
                rows = []
                error_message = " GC limiter specified but" 
                error_message +=" not the GC limit variable direction or the GC value."
            elif gc_value is not None and gc_limiter == 0 or gc_limit_type == 0:
                rows = []
                error_message = " GC value specified but" 
                error_message += " not the GC limit variable or the GC limiter."
                
            else: 
                    
                if gc_limiter == 1:
                
                    sql = '''CREATE TEMPORARY TABLE temp_repeats AS SELECT'''
                    sql += ''' id FROM repeats where gc < {0} AND repeat_seq != 'NO_REPEATS';'''.format(gc_value)
                elif gc_limiter == 2:
                    
                    sql = '''CREATE TEMPORARY TABLE temp_repeats AS SELECT''' 
                    sql += ''' id FROM repeats where gc > {0} AND repeat_seq != 'NO_REPEATS';'''.format(gc_value)
                elif gc_limiter == 3:
                    
                    sql = '''CREATE TEMPORARY TABLE temp_repeats AS SELECT'''
                    sql += ''' id FROM repeats where gc <= {0} AND repeat_seq != 'NO_REPEATS';'''.format(gc_value)
                elif gc_limiter == 4:
                    
                    sql = '''CREATE TEMPORARY TABLE temp_repeats AS SELECT'''
                    sql += ''' id FROM repeats where gc >= {0} AND repeat_seq != 'NO_REPEATS';'''.format(gc_value)
                elif gc_limiter == 5:
                    
                    sql = '''CREATE TEMPORARY TABLE temp_repeats AS SELECT'''
                    sql += ''' id FROM repeats where gc = {0} AND repeat_seq != 'NO_REPEATS';'''.format(gc_value)
                
                print("{0} Fetching data for GC={1}".format(INFO, gc_value))
                print("{0} Create temporary table sql={1}".format(INFO, sql))
                
                try:
                    
                    local_db = SQLiteDBConnector(db_file="../../play_ground.sqlite3")
                    local_db.connect()
                    local_db.create_tmp_table(sql=sql)
                    
                    sql = "SELECT COUNT(*) FROM temp_repeats"
                    rows = local_db.fetch_all(sql=sql)
                    
                    print("{0} Found {1} repeats with GC {2} {3}".format(INFO, rows[0][0], 
                                                                         gc_limit_map[gc_limiter],
                                                                         gc_value))
                    
                    sql = "SELECT value FROM repeats_distances WHERE"
                    sql += " repeat_idx_1 IN (SELECT * FROM temp_repeats)"
                    sql += " AND repeat_idx_2 IN (SELECT * FROM temp_repeats)"
                    sql += " AND hmm_state_id_1 = {0}".format(state_type_id)
                    sql += " AND  hmm_state_id_2 = {0}".format(state_type_id)
                    sql += " AND metric_type_id={0}".format(metric_type_id)
                    sql += " AND sequence_type_id={0}".format(sequence_type_id)

                    print("{0} Executing sql={1}".format(INFO, sql))
                    rows = local_db.fetch_all(sql=sql)
                    print("{0} Fetched number of rows={1}".format(INFO, len(rows)))
                    print("{0} Deleting table...".format(INFO))
                    sql = '''DROP TABLE IF EXISTS temp_repeats'''
                    local_db.execute_sql(sql=sql)
                    error_message = ""
                except Exception as e:
                    error_message = str(e)
                    rows = []
                    sql = '''DROP TABLE IF EXISTS temp_repeats'''
                    local_db.execute_sql(sql=sql)
                
    counts, bins = np.histogram(rows, bins=35)
    bins = 0.5 * (bins[:-1] + bins[1:])

    fig = px.bar(x=bins, y=counts, orientation='v', labels={'x':'distance', 'y':'count'}, range_x=[0,1])
    fig.update_layout(xaxis = dict(
            tickmode = 'linear',
            tick0 = 0.0,
            dtick = 0.15))

    return error_message, fig, len(rows)
    
    

In [18]:

@app.callback(
    Output("error-messages-id", component_property='children'),
    Output("normal-bar-chart", "figure"),
    Output("normal-n-distances", component_property='children'),
    Output("tuf-bar-chart", "figure"),
    Output("tuf-n-distances", component_property='children'),
    Output("core-bar-chart", "figure"),
    Output("core-n-distances", component_property='children'),
    [Input("dropdown-sequence", "value"), 
     Input("dropdown-distance", "value"),
     Input("dropdown-gc-limit-type", "value"),
     Input("dropdown-gc-limit", "value"),
     Input("gc-limit-value", "value"),
     Input("compute-btn", "n_clicks")],
     [State("compute-btn", "n_clicks")])
def update_bar_chart(seq_type, distance_type, gc_limit_type, 
                     gc_limiter, gc_value, btn_clicks, n_btn_clicks):
    
    # get the changes
    changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0]
    
    
    # if the compute bth is in the changes
    # this means we triger a compute
    if 'compute-btn' in changed_id:
        btn_clicks = 1
    else:
        btn_clicks = 0
    
    metric_type_id = distance_type           
    sequence_type_id = seq_type
    error_message = ""
    
    figs_ids = [1, 2, 3]
    figs = []
    for fid in figs_ids:
        error_message, fig, rows = create_figure_plot(state_type_id=fid, 
                                 metric_type_id=metric_type_id,
                                 sequence_type_id=sequence_type_id,
                                 gc_limit_type=gc_limit_type, 
                                 gc_limiter=gc_limiter, 
                                 gc_value=gc_value, btn_clicks=btn_clicks)
        figs.append(fig)
        figs.append(rows)
    
    
    return error_message, figs[0], figs[1], figs[2], figs[3], figs[4], figs[5],

In [19]:
# Run app and display result inline in the notebook
app.run_server(mode='external', use_reloader=False)

Dash app running on http://127.0.0.1:8050/
INFO: Fetching data for GC=0.6
INFO: Create temporary table sql=CREATE TEMPORARY TABLE temp_repeats AS SELECT id FROM repeats where gc < 0.6 AND repeat_seq != 'NO_REPEATS';
INFO: Found 8 repeats with GC < 0.6
INFO: Executing sql=SELECT value FROM repeats_distances WHERE repeat_idx_1 IN (SELECT * FROM temp_repeats) AND repeat_idx_2 IN (SELECT * FROM temp_repeats) AND hmm_state_id_1 = 1 AND  hmm_state_id_2 = 1 AND metric_type_id=1 AND sequence_type_id=1
INFO: Fetched number of rows=0
INFO: Deleting table...
INFO: Fetching data for GC=0.6
INFO: Create temporary table sql=CREATE TEMPORARY TABLE temp_repeats AS SELECT id FROM repeats where gc < 0.6 AND repeat_seq != 'NO_REPEATS';
INFO: Found 8 repeats with GC < 0.6
INFO: Executing sql=SELECT value FROM repeats_distances WHERE repeat_idx_1 IN (SELECT * FROM temp_repeats) AND repeat_idx_2 IN (SELECT * FROM temp_repeats) AND hmm_state_id_1 = 2 AND  hmm_state_id_2 = 2 AND metric_type_id=1 AND sequence_